In [ ]:
import numpy as np 
import pandas as pd
import math

# Creating Documents

In [ ]:
# reading dataset 
data = pd.read_csv('../input/craigslist-carstrucks-data/vehicles.csv')

In [ ]:
# checking total null values in each column 
data[data.columns].isna().sum()

In [ ]:
# droping unwandted columns county,id,drive,size,lat,long 
data = data.drop(['county','id','drive','size','lat','long'], axis = 1)

In [ ]:
# checking tha null values after droping unwanted columns 
data[data.columns].isna().sum()

In [ ]:
# keeping only the rows that do not have null values 
data = data[~data.isnull().any(axis=1)]

In [ ]:
data[data.columns].isna().sum()

In [ ]:
# total number of rows 
print(len(data.index))

In [ ]:
# printing data columns 
data.columns

In [ ]:
# converting to int attributes to strings

data["price"] = data["price"].map(str)
data["year"] = data["year"].map(str)
data["cylinders"] = data["cylinders"].map(str)
data["odometer"] = data["odometer"].map(str)
data["posting_date"] = data["posting_date"].map(str)

In [ ]:
# creaitng documents
data["Document"] = "listing URL : " + data["url"] + ", craigslist region : " + data["region"] + ", region URL : " + data["region_url"] + ", entry price : " + data["price"]  + ", entry year : " + data["year"] + ", manufacturer of vehicle : " + data["manufacturer"] + ", model of vehicle : " + data["model"] + ", condition of vehicle : " + data["condition"] + ", number of cylinders : " + data["cylinders"] + ", fuel type : " + data["fuel"] + ", miles traveled by vehicle : " +  data["odometer"] + ", title status of vehicle :" + data["title_status"] + ", transmission of vehicle :" + data["transmission"] + ", vehicle identification number : " + data["VIN"] + ", generic type of vehicle : " + data["type"] + ", color of vehicle : " + data["paint_color"] + ", image URL : " + data["image_url"] + ", listed description of vehicle : " + data["description"] + ", state of listing : " + data["state"] + ", posting_date : " + data["posting_date"]

In [ ]:
# droping unwandted columns county,id,drive,size,lat,long 
data = data.drop(['url', 'region', 'region_url', 'price', 'year', 'manufacturer', 'model','condition', 'cylinders', 'fuel', 'odometer', 'title_status','transmission', 'VIN', 'type', 'paint_color', 'image_url','description', 'state', 'posting_date'], axis = 1)

In [ ]:
# reseting the index
data = data.reset_index()
data = data.drop(["index"], axis = 1)

In [ ]:
data.head()

In [ ]:
# Printing 1 document
for index, row in data.iterrows():
    print(index)
    print(row['Document'])
    break

# Positional Indexing

In [ ]:
# importing necessary packages 
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string

In [ ]:
def preprocessing(final_string):
    # Tokenize.
    tokenizer = TweetTokenizer()
    token_list = tokenizer.tokenize(final_string)
     
    # Remove punctuations.
    table = str.maketrans('', '', '\t')
    token_list = [word.translate(table) for word in token_list]
    punctuations = (string.punctuation).replace("'", "")
    trans_table = str.maketrans('', '', punctuations)
    stripped_words = [word.translate(trans_table) for word in token_list]
    token_list = [str for str in stripped_words if str]
 
    # Change to lowercase.
    token_list =[word.lower() for word in token_list]
    
    # remove stop words
    stop_words = set(stopwords.words('english'))
    final_token_list = []
    
    for w in token_list:
        if w not in stop_words:
            final_token_list.append(w)
    
    return final_token_list

In [ ]:
def positionalIndex(data):
    # Initialize the dictionary.
    pos_index = {}
    stemmer = PorterStemmer()
    # For every file.
    for fileno, row in data.iterrows():
        doc = row["Document"]
        final_token_list = preprocessing(doc)

        for pos, term in enumerate(final_token_list):
            term = stemmer.stem(term)

            if term in pos_index:             
                # Increment total freq by 1.
                pos_index[term][0] = pos_index[term][0] + 1

                # Check if the term has existed in that DocID before.
                if fileno in pos_index[term][1]:
                    pos_index[term][1][fileno].append(pos)

                else:
                    pos_index[term][1][fileno] = [pos]
            else:

                # Initialize the list.
                pos_index[term] = []
                # The total frequency is 1.
                pos_index[term].append(1)
                # The postings list is initially empty.
                pos_index[term].append({})     
                # Add doc ID to postings list.
                pos_index[term][1][fileno] = [pos]
        if(fileno > 100):
            return pos_index

In [ ]:
# creaing index 
pos_index = positionalIndex(data)

In [ ]:
pos_index["messag"]

# TF-IDF ranking

In [ ]:
# selecting 100 docs in 71k 
data = data.head(100)

In [ ]:
# bag of words and unique words
bagOfWords = {} # all unique words present in each document
uniqueWords = set() # all unique words present in all document 
for fileno, row in data.iterrows():
    bagOfWordsDoc = preprocessing(row["Document"])
    bagOfWords[fileno] = bagOfWordsDoc
    uniqueWords = set(uniqueWords).union(set(bagOfWordsDoc))

In [ ]:
# total unique words
print(len(uniqueWords))

In [ ]:
# term frequency
tf = {}
for doc, words in bagOfWords.items():
    freq = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWords[doc]:
        freq[word] += 1  
    for w, f in freq.items():
        if(f > 0):
            freq[w] = 1 + math.log(f,10)
    tf[doc] = freq

In [ ]:
# inverse doc frequency

N = len(tf.keys())
idf = dict.fromkeys(tf[0].keys(), 0)
for doc in tf:
    for word, val in tf[doc].items():
        if val > 0:
            idf[word] += 1

for word, val in idf.items():
    idf[word] = math.log(N / float(val))

In [ ]:
# tf-idf
tfidf = []
for file in tf.keys():
    tfidfFile = {}
    for word, val in tf[file].items():
        tfidfFile[word] = val * idf[word]
    tfidf.append(tfidfFile) 

In [ ]:
# convert to dataFrame
tfidfDataFrame = pd.DataFrame(tfidf)

In [ ]:
# tf-idf 
tfidfDataFrame

In [ ]:
# cosine similarity function 

def cos(doc, qtfidf):
    q_dot_d = 0
    modD = 0
    modQ = 0
    for word in uniqueWords:
        q_dot_d = q_dot_d + (doc[word] * qtfidf[word])
    for word,val in doc.items():
        modD = modD + (val*val)
    for word,val in qtfidf.items():
        modQ = modQ + (val*val)
    return q_dot_d/(modD*modQ)

In [ ]:
# tf-idf for query

# query
q = "entry price : 33590, entry year : 2014.0, manufacturer of vehicle : gmc, model of vehicle : sierra 1500 crew cab slt"
q_token = preprocessing(q)
# tf of q
qtf = dict.fromkeys(uniqueWords, 0)
for word in q_token:
    qtf[word] += 1
for w, f in qtf.items():
    if(f > 0):
        qtf[w] = 1 + math.log(f,10)

# tf-idf of q
for file in tf.keys():
    qtfidf = {}
    for word, val in qtf.items():
        qtfidf[word] = val * idf[word]


In [ ]:
# retrieve top 10 releven doc for given query

cosineSim = {}
i = 0
for doc in tfidf:
    cosineSim[i] = cos(doc, qtfidf)
    i = i+1

tfidfRanking = dict(sorted(cosineSim.items(), key=lambda x: x[1], reverse=True))
ans = {k: tfidfRanking[k] for k in list(tfidfRanking)[:10]}
rank = 1
for docID, cosScore in ans.items():
    print("Rank : ", rank, ", docID : ", docID, ", cosScore : ", cosScore)
    rank = rank + 1

# Evaluation

In [ ]:
q = "entry price : 33590, entry year : 2014.0, manufacturer of vehicle : gmc, model of vehicle : sierra 1500 crew cab slt"
totalReleventDoc = 5
releventDoc = [1, 2, 5, 7, 10] 
rank = 1
for docID, cosScore in ans.items():
    if rank in releventDoc:
        print("Rank : ", rank, ", docID : ", docID, ", relevant doc")
    else:
        print("Rank : ", rank, ", docID : ", docID, ", Not relevant doc")
    rank = rank + 1

In [ ]:
# precision and recall 

recall = []
precision = []

rank = 1
relevalntRetrived = 0
for docID, cosScore in ans.items():
    if rank in releventDoc:
        relevalntRetrived = relevalntRetrived + 1
    recall.append(relevalntRetrived/totalReleventDoc)
    precision.append(relevalntRetrived/rank)
    print("Rank : ", rank, ", recall : ", recall[rank-1], ", precision : ",precision[rank-1])
    rank = rank + 1


In [ ]:
# 11 point interpolation 
# [0.0 , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
precision.reverse()

for i in range(1,len(precision),1):
    precision[i] = max(precision[i-1],precision[i])
precision.reverse()
print(precision)

In [ ]:
# 11pt interpolation of precision and recall
import matplotlib.pyplot as plt
  
plt.plot(recall, precision,'bo')
plt.plot(recall, precision, color = "red")
plt.title('precision Vs recall')
plt.xlabel('recall')
plt.ylabel('precision')
plt.show()

# Relvance feedback

In [ ]:
q = "entry price : 33590, entry year : 2014.0, manufacturer of vehicle : gmc, model of vehicle : sierra 1500 crew cab slt"
totalReleventDoc = 5
releventDoc = [1, 2, 5, 7, 10]
releventDocID = [0, 57, 30, 58, 47]
rank = 1
for docID, cosScore in ans.items():
    if rank in releventDoc:
        print("Rank : ", rank, ", docID : ", docID, ", relevant doc")
    else:
        print("Rank : ", rank, ", docID : ", docID, ", Not relevant doc")
    rank = rank + 1

In [ ]:
# calculate centroids
import numpy as np

Rcentroid = np.zeros(len(uniqueWords))
NRcentroid = np.zeros(len(uniqueWords))
rank = 1
for doc in ans:
    if rank in releventDoc:
        i = 0
        for word, val in tfidf[doc].items():
            Rcentroid[i] = Rcentroid[i] + val
    else:
        i = 0
        for word, val in tfidf[doc].items():
            NRcentroid[i] = NRcentroid[i] + val
    rank = rank + 1
    
Rcentroid = np.true_divide(Rcentroid, 4)
NRcentroid = np.true_divide(NRcentroid, 4)

In [ ]:
# update query using rocchio algorithm
Q = np.zeros(len(uniqueWords))
i = 0
for val in qtfidf:
    Q[i] = qtfidf[val]
    i = i + 1

alpha = 1
beta = 0.75
gama = 0.25
newQ = alpha * Q  + ((beta * Rcentroid) - (gama * NRcentroid))

newqtfidf = {}
i = 0
for val in qtfidf:
    newqtfidf[val] = newQ[i]
    i = i + 1

In [ ]:
# retrieve top 10 releven doc for updated query

cosineSim = {}
i = 0
for doc in tfidf:
    cosineSim[i] = cos(doc, newqtfidf)
    i = i+1

tfidfRanking = dict(sorted(cosineSim.items(), key=lambda x: x[1], reverse=True))
ans = {k: tfidfRanking[k] for k in list(tfidfRanking)[:10]}
rank = 1
for docID, cosScore in ans.items():
    print("Rank : ", rank, ", docID : ", docID, ", cosScore : ", cosScore)
    rank = rank + 1


In [ ]:
q = "entry price : 33590, entry year : 2014.0, manufacturer of vehicle : gmc, model of vehicle : sierra 1500 crew cab slt"
totalReleventDoc = 5

releventDocID = [0, 57, 30, 58, 47]
rank = 1
for docID, cosScore in ans.items():
    if docID in releventDocID:
        print("Rank : ", rank, ", docID : ", docID, ", relevant doc")
    else:
        print("Rank : ", rank, ", docID : ", docID, ", Not relevant doc")
    rank = rank + 1